In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from PIL import Image
import torch.nn.functional as F

In [2]:
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
from nuscenes.nuscenes import NuScenes
PATH = '/Users/jonathanmorris/Downloads/v1.0-mini'

nusc = NuScenes(version='v1.0-mini', dataroot=PATH, verbose=True)
nusc_can = NuScenesCanBus(dataroot=PATH)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.574 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
nusc.list_scenes()

scene-0061, Parked truck, construction, intersectio... [18-07-24 03:28:47]   19s, singapore-onenorth, #anns:4622
scene-0103, Many peds right, wait for turning car, ... [18-08-01 19:26:43]   19s, boston-seaport, #anns:2046
scene-0655, Parking lot, parked cars, jaywalker, be... [18-08-27 15:51:32]   20s, boston-seaport, #anns:2332
scene-0553, Wait at intersection, bicycle, large tr... [18-08-28 20:48:16]   20s, boston-seaport, #anns:1950
scene-0757, Arrive at busy intersection, bus, wait ... [18-08-30 19:25:08]   20s, boston-seaport, #anns:592
scene-0796, Scooter, peds on sidewalk, bus, cars, t... [18-10-02 02:52:24]   20s, singapore-queensto, #anns:708
scene-0916, Parking lot, bicycle rack, parked bicyc... [18-10-08 07:37:13]   20s, singapore-queensto, #anns:2387
scene-1077, Night, big street, bus stop, high speed... [18-11-21 11:39:27]   20s, singapore-hollandv, #anns:890
scene-1094, Night, after rain, many peds, PMD, ped ... [18-11-21 11:47:27]   19s, singapore-hollandv, #anns:1762
sc

In [4]:
# Selecting first scene
my_scene = nusc.scene[0]

# Getting data from can expansion
my_scene_can = nusc_can.get_messages(my_scene['name'], 'zoesensors')
my_scene_can[0]

{'brake_sensor': 0.18794798851013184,
 'steering_sensor': 0.18827444314956665,
 'throttle_sensor': 0.12093144655227661,
 'utime': 1532402927649559}

In [5]:
# Get first sample
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)
my_sample

{'token': 'ca9a282c9e77460f8360f564131a8af5',
 'timestamp': 1532402927647951,
 'prev': '',
 'next': '39586f9d59004284a7114a68825e8eec',
 'scene_token': 'cc8c0bf57f984915a77078b10eb33198',
 'data': {'RADAR_FRONT': '37091c75b9704e0daa829ba56dfa0906',
  'RADAR_FRONT_LEFT': '11946c1461d14016a322916157da3c7d',
  'RADAR_FRONT_RIGHT': '491209956ee3435a9ec173dad3aaf58b',
  'RADAR_BACK_LEFT': '312aa38d0e3e4f01b3124c523e6f9776',
  'RADAR_BACK_RIGHT': '07b30d5eb6104e79be58eadf94382bc1',
  'LIDAR_TOP': '9d9bf11fb0e144c8b446d54a8a00184f',
  'CAM_FRONT': 'e3d495d4ac534d54b321f50006683844',
  'CAM_FRONT_RIGHT': 'aac7867ebf4f446395d29fbd60b63b3b',
  'CAM_BACK_RIGHT': '79dbb4460a6b40f49f9c150cb118247e',
  'CAM_BACK': '03bea5763f0f4722933508d5999c5fd8',
  'CAM_BACK_LEFT': '43893a033f9c46d4a51b5e08a67a1eb7',
  'CAM_FRONT_LEFT': 'fe5422747a7d4268a4b07fc396707b23'},
 'anns': ['ef63a697930c4b20a6b9791f423351da',
  '6b89da9bf1f84fd6a5fbe1c3b236f809',
  '924ee6ac1fed440a9d9e3720aac635a0',
  '91e3608f55174a319

In [6]:
print(nusc_can.can_blacklist)
print(nusc.scene[0]['name'])
print(int(nusc.scene[0]['name'].split("-")[1]))

[161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 172, 173, 174, 175, 176, 309, 310, 311, 312, 313, 314]
scene-0061
61


In [7]:
def get_closest_can(time, can_objects):
    closest = {}
    prev_diff = 1000000000000000000000000
    for object in can_objects:
        diff = object["utime"] - time
        if diff > 0 and diff < prev_diff:
            closest = object
            prev_diff = diff
    return closest

def normalize_can(can_obj):
    new_obj = can_obj
    
    # These values are from here: https://github.com/nutonomy/nuscenes-devkit/blob/master/python-sdk/nuscenes/can_bus/README.md#zoe-sensors
    min_breaking = 0.166
    max_breaking = 0.631
    min_steering = 0.176
    max_steering = 0.252
    min_throttle = 0.105
    max_throttle = 0.411

    # convert values from 0-1
    new_obj["brake_sensor"] = (can_obj["brake_sensor"] - min_breaking) / (max_breaking - min_breaking)
    new_obj["steering_sensor"] = (can_obj["steering_sensor"] - min_steering) / (max_steering - min_steering)
    new_obj["throttle_sensor"] = (can_obj["throttle_sensor"] - min_throttle) / (max_throttle - min_throttle)

    # reduce precision
    new_obj["brake_sensor"] = round(can_obj["brake_sensor"], 2)
    new_obj["steering_sensor"] = round(can_obj["steering_sensor"], 2)
    new_obj["throttle_sensor"] = round(can_obj["throttle_sensor"], 2)
    
    return new_obj
    

In [8]:
can_data = get_closest_can(my_sample['timestamp'], my_scene_can)
can_data


{'brake_sensor': 0.18794798851013184,
 'steering_sensor': 0.18827444314956665,
 'throttle_sensor': 0.12093144655227661,
 'utime': 1532402927649559}

In [9]:
normalize_can(can_data)

{'brake_sensor': 0.05,
 'steering_sensor': 0.16,
 'throttle_sensor': 0.05,
 'utime': 1532402927649559}

In [10]:
next_sample = nusc.get('sample', my_sample['next'])
can_data = get_closest_can(next_sample['timestamp'], my_scene_can)
can_data

{'brake_sensor': 0.20549276471138,
 'steering_sensor': 0.18824294209480286,
 'throttle_sensor': 0.1209488958120346,
 'utime': 1532402928147850}

In [11]:
normalize_can(can_data)

{'brake_sensor': 0.08,
 'steering_sensor': 0.16,
 'throttle_sensor': 0.05,
 'utime': 1532402928147850}

In [12]:
next2_sample = nusc.get('sample', next_sample['next'])
can_data = get_closest_can(next2_sample['timestamp'], my_scene_can)
can_data

{'brake_sensor': 0.25458383560180664,
 'steering_sensor': 0.18891645967960358,
 'throttle_sensor': 0.1220652386546135,
 'utime': 1532402928701206}

In [13]:
normalize_can(can_data)

{'brake_sensor': 0.19,
 'steering_sensor': 0.17,
 'throttle_sensor': 0.06,
 'utime': 1532402928701206}

In [14]:
can_data = get_closest_can(my_sample['timestamp'], my_scene_can)
print("Before: " + str(can_data))
normal_can = normalize_can(can_data)
print("After: " + str(can_data))

Before: {'brake_sensor': 0.05, 'steering_sensor': 0.16, 'throttle_sensor': 0.05, 'utime': 1532402927649559}
After: {'brake_sensor': -0.25, 'steering_sensor': -0.21, 'throttle_sensor': -0.18, 'utime': 1532402927649559}


In [15]:
sensor = 'CAM_FRONT'
cam_front_data = nusc.get('sample_data', my_sample['data'][sensor])
current_image_path = PATH+"/"+cam_front_data['filename']

In [16]:
img = Image.open(current_image_path)
convert_tensor = transforms.ToTensor()
convert_tensor(img)

tensor([[[0.1216, 0.1176, 0.1098,  ..., 0.4863, 0.4863, 0.4863],
         [0.1176, 0.1176, 0.1137,  ..., 0.4863, 0.4863, 0.4863],
         [0.0980, 0.1020, 0.1020,  ..., 0.4863, 0.4863, 0.4863],
         ...,
         [0.1020, 0.2431, 0.2627,  ..., 0.4000, 0.4039, 0.4039],
         [0.1020, 0.2431, 0.2627,  ..., 0.3961, 0.3961, 0.4000],
         [0.1020, 0.2431, 0.2627,  ..., 0.3922, 0.3961, 0.3961]],

        [[0.0863, 0.0824, 0.0863,  ..., 0.5412, 0.5412, 0.5412],
         [0.0824, 0.0824, 0.0902,  ..., 0.5412, 0.5412, 0.5412],
         [0.0627, 0.0667, 0.0784,  ..., 0.5412, 0.5412, 0.5412],
         ...,
         [0.1020, 0.2431, 0.2627,  ..., 0.4000, 0.4039, 0.4039],
         [0.1020, 0.2431, 0.2627,  ..., 0.3961, 0.3961, 0.4000],
         [0.1020, 0.2431, 0.2627,  ..., 0.3922, 0.3961, 0.3961]],

        [[0.0980, 0.0941, 0.1020,  ..., 0.5922, 0.5922, 0.5922],
         [0.0941, 0.0941, 0.1059,  ..., 0.5922, 0.5922, 0.5922],
         [0.0667, 0.0706, 0.0863,  ..., 0.5922, 0.5922, 0.

In [17]:
def get_next_data(scene, sample):
    next_token = sample["next"]
    next_sample = nusc.get("sample", next_token)
    can_data = nusc_can.get_messages(scene['name'], 'zoesensors')
    get_closest_can(next_sample["timestamp"], my_scene_can)
    return (next_sample, can_data)

get_next_data(my_scene,my_sample)

({'token': '39586f9d59004284a7114a68825e8eec',
  'timestamp': 1532402928147847,
  'prev': 'ca9a282c9e77460f8360f564131a8af5',
  'next': '356d81f38dd9473ba590f39e266f54e5',
  'scene_token': 'cc8c0bf57f984915a77078b10eb33198',
  'data': {'RADAR_FRONT': 'b70cefb08263499eb30c7e7da0031428',
   'RADAR_FRONT_LEFT': 'b5cd02b25f0944f19c0d123fa1fc54ec',
   'RADAR_FRONT_RIGHT': '9b628ce4952a4a839962b77a76630e23',
   'RADAR_BACK_LEFT': '28b610ada2b545d1a5a1bf9d08af0285',
   'RADAR_BACK_RIGHT': '07a62118eb3b43ffa1d833f250ff6f80',
   'LIDAR_TOP': '4f792c8da81e4cb7aca1790654da1c27',
   'CAM_FRONT': '4b6870ae200c4b969b91c50a9737f712',
   'CAM_FRONT_RIGHT': '5c026763ad8146e89c6ee53da26331dc',
   'CAM_BACK_RIGHT': 'c339f7629fdf4c219e30ca1790b53773',
   'CAM_BACK': '661d9842bbb44b208f867f2061c4f535',
   'CAM_BACK_LEFT': '9320f00a62c24b80972048de7a742b4c',
   'CAM_FRONT_LEFT': '2b9d52c5c777455eb31831860c6fc117'},
  'anns': ['7987617983634b119e383d8a29607fd7',
   '216bbbd8e01c450a8fabe9d47433c10a',
   'f0c

In [18]:
class LaneCNN(nn.Module):
    def __init__(self):
        super(LaneCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(256 * 28 * 28, 512)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(512, 3) # output three values -> [steering, throttle, breaking]

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = self.flatten(x)
        x = x.view(256* 28*28)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class LaneCNNv2(nn.Module):
    def __init__(self):
        super(LaneCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)  # Adding dropout for regularization

        self.fc1 = nn.Linear(256 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 3)  # Output three values -> [steering, throttle, braking]

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.dropout(x)
        x = x.view(-1, 256 * 28 * 28)  # Reshape the tensor for fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [22]:
learning_rate = 0.001
batch_size = 64
epochs = 10
model_path = os.path.join(os.getcwd(), "models")
!mkdir models && cd models mkdir epochs

zsh:cd:1: too many arguments


In [20]:
def train():
    # path = sys.argv[1]
    print("final model weights will be saved to: " + model_path)

    device = torch.device("cuda")
    
    transform = transforms.Compose(
        [transforms.Resize((224, 224), antialias=True), transforms.ToTensor()]
    )

    scenes = nusc.scene

    train_size = int(0.8 * len(scenes))
    val_size = len(scenes) - train_size
    train, val = random_split(scenes, [train_size, val_size])

    # multithreaded data loading
    trainloader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)

    valloader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=2)

    net = LaneCNN().to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    print(f"Training on {device}")

    for epoch in range(epochs):
        net.train()
        for scene in trainloader:
            for scene in scenes:
                first_sample_token = scene['first_sample_token']

                current_sample = nusc.get('sample', first_sample_token)
                scene_cans = nusc_can.get_messages(scene['name'], 'zoesensors')

                scene_number = int(scene['name'].split("-")[1])

                if scene_number in nusc_can.can_blacklist:
                    print("Skipping scene " + str(scene_number))
                    continue

                while True:
                    sensor = "CAM_FRONT"
                    cam_front_data = nusc.get("sample_data", current_sample["data"][sensor])
                    current_image_path = PATH + "/" + cam_front_data["filename"]
                    img = Image.open(current_image_path)

                    inputs = transform(img).to(device)
                    
                    current_can = get_closest_can(current_sample["timestamp"], scene_cans)

                    normal_can = normalize_can(current_can)

                    steering_targets = normal_can['steering_sensor']
                    throttle_targets = normal_can['throttle_sensor']
                    breaking_targets = normal_can['brake_sensor']


                    label = torch.FloatTensor([steering_targets, throttle_targets, breaking_targets]).to(device)

                    optimizer.zero_grad()
                    

                    # Forward pass
                    outputs = net(inputs)

                    # Compute loss
                    total_loss = criterion(outputs, label)

                    # Backward pass
                    total_loss.backward()

                    # Update weights
                    optimizer.step()

                    if current_sample['next'] == '':
                        break
                    else:
                        current_sample = nusc.get('sample', current_sample['next'])

        # Validation
        net.eval()
        with torch.no_grad():
            for scene in trainloader:
                for scene in scenes:
                    first_sample_token = scene['first_sample_token']

                    current_sample = nusc.get('sample', first_sample_token)
                    scene_cans = nusc_can.get_messages(scene['name'], 'zoesensors')

                    scene_number = int(scene['name'].split("-")[1])

                    if scene_number in nusc_can.can_blacklist:
                        print("Skipping scene " + str(scene_number))
                        continue

                    while True:
                        sensor = "CAM_FRONT"
                        cam_front_data = nusc.get("sample_data", current_sample["data"][sensor])
                        current_image_path = PATH + "/" + cam_front_data["filename"]
                        img = Image.open(current_image_path)

                        inputs = transform(img).to(device)

                        current_can = get_closest_can(current_sample["timestamp"], scene_cans)
                        
                        normal_can = normalize_can(current_can)

                        steering_targets = normal_can['steering_sensor']
                        throttle_targets = normal_can['throttle_sensor']
                        breaking_targets = normal_can['brake_sensor']

                        label = torch.FloatTensor([steering_targets, throttle_targets, breaking_targets]).to(device)

                        outputs = net(inputs)

                        val_total_loss = criterion(outputs, label)

                        if current_sample['next'] == '':
                            break
                        else:
                            current_sample = nusc.get('sample', current_sample['next'])

        print(
            f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss.item():.4f}, Validation Loss: {val_total_loss.item():.4f}"
        )
        torch.save(
            net.state_dict(),
            os.path.join(model_path, "epochs", f"model_e{epoch+1}.pth"),
        )

    print("Finished training")
    torch.save(net.state_dict(), os.path.join(model_path, f"model_final.pth"))

In [ ]:
train()